**Zadanie 1. (5p)** Dodaj do wyszukiwarki Wikipedyjki indeks pozycyjny. Każde zapytanie powinno
być traktowane jako pytanie o frazę, przy czym powinieneś obsługiwać odmianę, tzn. pytanie
_skoki narciarskie_ powinno zwracać również dokumenty zawierające frazę _skoków narciarskich_. W
wypisywanych fragmentach dokumentów powinny być wyróżnione (najlepiej kolorem) trafienia całej
frazy. Kolejność wypisywania dokumentów może być dowolna.

In [1]:
import csv
import logging
from collections import defaultdict
from bisect import bisect

In [2]:
BASE_FORMS_FILE_PATH = 'data/polimorfologik-2.1.txt'
WIKI_ARTICLES_FILE_PATH = 'data/fp_wiki.txt'

In [3]:
BASE_FORMS = {}

with open(BASE_FORMS_FILE_PATH) as f:
    for base_form, word, *_ in csv.reader(f, delimiter=';'):
        BASE_FORMS[word.lower()] = base_form

In [4]:
def base(word):
    return BASE_FORMS.get(word.lower())

In [5]:
POSITION_BY_ID = []
WIKI_ARTICLES = []
POSITIONAL_INDEX = defaultdict(set)

with open(WIKI_ARTICLES_FILE_PATH) as f:
    lines = iter(f)
    position = 0
    try:
        while True:
            POSITION_BY_ID.append(position)
            _title_with_prefix = next(lines)
            title = next(lines).split()
            for word in title:
                POSITIONAL_INDEX[base(word)].add(position)
                position += 1
            text = []
            while sentence := next(lines).split():
                for word in sentence:
                    POSITIONAL_INDEX[base(word)].add(position)
                    position += 1
                text.extend(sentence)
            WIKI_ARTICLES.append((title, text))
    except StopIteration:
        pass

In [6]:
def get_id(position):
    return bisect(POSITION_BY_ID, position) - 1

In [7]:
def _find_positions(base_phrase):
    return set.intersection(*(
        {position - offset for position in POSITIONAL_INDEX[word]}
        for offset, word in enumerate(base_phrase)
    ))

In [8]:
def find_articles(phrase):
    base_phrase = [base(word) for word in phrase]
    if not all(base_phrase):
        raise ValueError(f'Could not find all base forms for "{phrase}"')
    articles_hits = defaultdict(set)
    for position in _find_positions(base_phrase):
        articles_hits[get_id(position)].add(position)
    results = []
    for id_, positions in articles_hits.items():
        start = POSITION_BY_ID[id_]
        title, text = WIKI_ARTICLES[id_]
        hits = [position - start for position in positions]
        results.append((
            (title, [hit for hit in hits if hit < len(title)]),
            (text, [hit - len(title) for hit in hits if hit >= len(title)]),
        ))
    return results

In [20]:
def highlight(text):
    return f'\033[1m\033[34m{text}\033[m'


def display(result, query_length):
    (title, title_hits), (text, text_hits) = result
    title_copy = title[:]
    text_copy = text[:]
    for hit in title_hits:
        title_copy[hit : hit + query_length] = [
            highlight(title_copy[hit + offset]) for offset in range(query_length)
        ]
    for hit in text_hits:
        text_copy[hit : hit + query_length] = [
            highlight(text_copy[hit + offset]) for offset in range(query_length)
        ]
    return f"{' '.join(title_copy)}\n{' '.join(text_copy)}\n\n"
    

def search(query, max_num_results=1000):
    phrase = query.split()
    results = find_articles(phrase)
    for result, _ in zip(results, range(max_num_results)):
        print(display(result, len(phrase)))

In [21]:
search('skoki narciarskie', max_num_results=10)

Johan Remen Evensen
Johan Remen Evensen ( ur . 16 września 1985 w Alsvågu ) – norweski skoczek narciarski pochodzący z Øksnes . Były rekordzista świata w długości skoku ( 246,5 m ) . W Pucharze Świata zadebiutował w sezonie 2008/2009 podczas konkursów w Trondheim , gdzie zajął 10. i 12. miejsce . Ze względu na dobry występ wziął udział w kolejnych konkursach we włoskim Pragelato , gdzie w drugim konkursie zajął 3. miejsce . Po raz kolejny na podium stanął 2 miesiące później w konkursie na mamuciej skoczni w Oberstdorfie . W 2009 uczestniczył w mistrzostwach świata w Libercu . W konkursach indywidualnych zajął 27. i 30. miejsce . W konkursie drużynowym zdobył srebrny medal . Cały sezon zakończył na 20. miejscu w klasyfikacji generalnej PŚ . W kolejnym sezonie 2009/2010 ustanowił swój nowy rekord życiowy 224,5 m w Oberstdorfie podczas kwalifikacji do konkursu indywidualnego 29 stycznia 2010 . Dwa dni później po raz trzeci w karierze stanął na podium , dokładnie na jego trzecim stopniu . 